# 第三章 索引

In [ ]:
import numpy as np
import pandas as pd

## 一、索引器
### 1. 表的列索引
列索引是最常见的索引形式，一般通过`[]`来实现。通过`[列名]`可以从`DataFrame`中取出相应的列，返回值为`Series`，例如从表中取出姓名一列：

In [ ]:
df = pd.read_csv('../data/learn_pandas.csv', usecols =['School', 'Grade', 'Name', 'Gender', 'Weight', 'Transfer'])
df['Name'].head()

如果要取出多个列，则可以通过`[列名组成的列表]`，其返回值为一个`DataFrame`，例如从表中取出性别和姓名两列：

In [ ]:
df[['Gender', 'Name']].head()

此外，若要取出单列，且列名中不包含空格，则可以用`.列名`取出，这和`[列名]`是等价的：

In [ ]:
df.Name.head()

### 2. 序列的行索引
【a】以字符串为索引的`Series`

如果取出单个索引的对应元素，则可以使用`[item]`，若`Series`只有单个值对应，则返回这个标量值，如果有多个值对应，则返回一个`Series`:

In [ ]:
s = pd.Series([1, 2, 3, 4, 5, 6], index=['a', 'b', 'a', 'a', 'a', 'c'])
s['a']

In [ ]:
s['b']

如果取出多个索引的对应元素，则可以使用`[items的列表]`：

In [ ]:
s[['c', 'b']]

如果想要取出某两个索引之间的元素，并且这两个索引是在整个索引中唯一出现，则可以使用切片,，同时需要注意这里的切片会包含两个端点：

In [ ]:
s['c':'b':-2]

如果前后端点的值重复出现，那么需要经过排序才能使用切片：

In [ ]:
try:
    s['a':'b']
except Exception as e:
    Err_Msg = e
Err_Msg

In [ ]:
s.sort_index()['a':'b']

【b】以整数为索引的`Series`

在使用数据的读入函数时，如果不特别指定所对应的列作为索引，那么会生成从0开始的整数索引作为默认索引。当然，任意一组符合长度要求的整数都可以作为索引。

和字符串一样，如果使用`[int]`或`[int_list]`，则可以取出对应索引元素的值：

In [ ]:
s = pd.Series(['a', 'b', 'c', 'd', 'e', 'f'], index=[1, 3, 1, 2, 5, 4])
s[1]

In [ ]:
s[[2, 3]]

如果使用整数切片，则会取出对应索引位置的值，注意这里的整数切片同`Python`中的切片一样不包含右端点：

In [ ]:
s[1:-1:2]

#### 【WARNING】关于索引类型的说明

如果不想陷入麻烦，那么请不要把纯浮点以及任何混合类型（字符串、整数、浮点类型等的混合）作为索引，否则可能会在具体的操作时报错或者返回非预期的结果，并且在实际的数据分析中也不存在这样做的动机。

### 3. loc索引器
前面讲到了对`DataFrame`的列进行选取，下面要讨论其行的选取。对于表而言，有两种索引器，一种是基于元素的`loc`索引器，另一种是基于位置的`iloc`索引器。

`loc`索引器的一般形式是`loc[*, *]`，其中第一个`*`代表行的选择，第二个`*`代表列的选择，如果省略第二个位置写作`loc[*]`，这个`*`是指行的筛选。其中，`*`的位置一共有五类合法对象，分别是：单个元素、元素列表、元素切片、布尔列表以及函数，下面将依次说明。

为了演示相应操作，先利用`set_index`方法把`Name`列设为索引，关于该函数的其他用法将在多级索引一章介绍。

In [ ]:
df_demo = df.set_index('Name')
df_demo.head()

【a】`*`为单个元素
此时，直接取出相应的行或列，如果该元素在索引中重复则结果为`DataFrame`，否则为`Series`:

In [ ]:
df_demo.loc['Qiang Sun'] # 多个人叫此名字

In [ ]:
df_demo.loc['Quan Zhao'] # 名字唯一

也可以同时选择行和列：

In [ ]:
df_demo.loc['Qiang Sun', 'School'] # 返回Series

In [ ]:
df_demo.loc['Quan Zhao', 'School'] # 返回单个元素

【b】`*`为元素列表

此时，取出列表中所有元素值对应的行或列：

In [ ]:
df_demo.loc[['Qiang Sun', 'Quan Zhao'], ['School', 'Gender']]

【c】`*`为切片

之前的`Series`使用字符串索引时提到，如果是唯一值的起点和终点字符，那么就可以使用切片，并且包含两个端点，如果不唯一则报错：

In [ ]:
df_demo.loc['Gaojuan You':'Gaoqiang Qian', 'School':'Gender']

需要注意的是，如果`DataFrame`使用整数索引，其使用整数切片的时候和上面字符串索引的要求一致，都是元素切片，包含端点且起点、终点不允许有重复值。

In [ ]:
df_loc_slice_demo = df_demo.copy()
df_loc_slice_demo.index = range(df_demo.shape[0], 0, -1)
df_loc_slice_demo.loc[5:3]

In [ ]:
df_loc_slice_demo.loc[3:5] # 没有返回，说明不是整数位置切片

【d】`*`为布尔列表

在实际的数据处理中，根据条件来筛选行是极其常见的，此处传入`loc`的布尔列表与`DataFrame`长度相同，且列表为`True`的位置所对应的行会被选中，`False`则会被剔除。

例如，选出体重超过70kg的学生：

In [ ]:
df_demo.loc[df_demo.Weight>70].head()

前面所提到的传入元素列表，也可以通过`isin`方法返回的布尔列表等价写出，例如选出所有大一和大四的同学信息：

In [ ]:
df_demo.loc[df_demo.Grade.isin(['Freshman', 'Senior'])].head()

对于复合条件而言，可以用`|`（或）, `&`（且）, `~`（取反）的组合来实现，例如选出复旦大学中体重超过70kg的大四学生，或者北大男生中体重超过80kg的非大四的学生：

In [ ]:
condition_1_1 = df_demo.School == 'Fudan University'
condition_1_2 = df_demo.Grade == 'Senior'
condition_1_3 = df_demo.Weight > 70
condition_1 = condition_1_1 & condition_1_2 & condition_1_3
condition_2_1 = df_demo.School == 'Peking University'
condition_2_2 = df_demo.Grade == 'Senior'
condition_2_3 = df_demo.Weight > 80
condition_2 = condition_2_1 & (~ condition_2_2) & condition_2_3
df_demo.loc[condition_1 | condition_2]


【e】`*`为函数
这里的函数，必须以前面的四种合法形式之一为返回值，并且函数的输入值为`DataFrame`本身。假设仍然是上述复合条件筛选的例子，可以把逻辑写入一个函数中再返回，需要注意的是函数的形式参数`x`本质上即为`df_demo`:

In [ ]:
def condition(x):
    condition_1_1 = x.School == 'Fudan University'
    condition_1_2 = x.Grade == 'Senior'
    condition_1_3 = x.Weight > 70
    condition_1 = condition_1_1 & condition_1_2 & condition_1_3
    condition_2_1 = x.School == 'Peking University'
    condition_2_2 = x.Grade == 'Senior'
    condition_2_3 = x.Weight > 80
    condition_2 = condition_2_1 & (~ condition_2_2) & condition_2_3
    result = condition_1 | condition_2

    return result

df_demo.loc[condition]


此外，还支持使用`lambda`表达式，其返回值也同样必须是先前提到的四种形式之一：

In [ ]:
df_demo.loc[lambda x:'Quan Zhao', lambda x:'Gender']

由于函数无法返回如`start: end: step`的切片形式，故返回切片时要用`slice`对象进行包装：

In [ ]:
df_demo.loc[lambda x:slice('Gaojuan You', 'Gaoqiang Qian')]

最后需要指出的是，对于`Series`也可以使用`loc`索引，其遵循的原则与`DataFrame`中用于行筛选的`loc[*]`完全一致。

#### 【WARNING】不要使用链式赋值
在对表或者序列赋值时，应当在使用一层索引器后直接进行赋值操作，这样做是由于进行多次索引后赋值是赋在临时返回的`copy`副本上的，而没有真正修改元素从而报出`SettingWithCopyWarning`警告。例如，下面给出的例子：

In [ ]:
df_chain = pd.DataFrame([[0,0],[1,0],[-1,0]], columns=list('AB'))
df_chain
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings('error')
    try:
        df_chain[df_chain.A!=0].B = 1  # 使用方括号列索引后，再使用点的列索引
    except Warning as w:
        Warning_Msg = w

print(Warning_Msg)
df_chain

In [ ]:
df_chain.loc[df_chain.A!=0, 'B'] = 1
df_chain

### 4. `iloc`索引器
`iloc`的使用与`loc`完全类似，只不过是针对位置进行筛选，在相应的`*`位置处一共也有五类合法对象，分别是：整数、整数列表、整数切片、布尔列表以及函数，函数的返回值必须是前面的四类合法对象中的一个，其输入同样也为`DataFrame`本身。

In [ ]:
df_demo.iloc[1, 1] # 第二行第二列

In [ ]:
df_demo.iloc[[0, 1], [0, 1]] # 前两行前两列

In [ ]:
df_demo.iloc[1:4, 2:4] # 切片不包含结束端点

In [ ]:
df_demo.iloc[lambda x:slice(1, 4)] # 传入切片为返回值的函数

在使用布尔列表的时候要特别注意，不能传入`Series`而必须传入序列的`values`，否则会报错。因此，在使用布尔筛选的时候还是应当优先考虑`loc`的方式。

例如，选出体重超过80kg的学生：

In [ ]:
df_demo.iloc[(df_demo.Weight>80).values].head()

对`Series`而言同样也可以通过`iloc`返回相应位置的值或子序列：

In [ ]:
df_demo.School.iloc[1]

In [ ]:
df_demo.School.iloc[1:5:2]

### 5. `query`方法

在`pandas`中，支持把字符串形式的查询表达式传入`query`方法来查询数据，其表达式的执行结果必须返回布尔列表。在进行复杂索引时，由于这种检索方式无需像普通方法一样重复使用`DataFrame`的名字来引用列名，一般而言会使代码长度在不降低可读性的前提下有所减少。

例如，将`loc`一节中的复合条件查询例子可以如下改写：


In [ ]:
df.query('((School == "Fudan University")&'
         ' (Grade == "Senior")&'
         ' (Weight > 70))|'
         '((School == "Peking University")&'
         ' (Grade != "Senior")&'
         ' (Weight > 80))')

在`query`表达式中，帮用户注册了所有来自`DataFrame`的列名，所有属于该`Series`的方法都可以被调用，和正常的函数调用并没有区别，例如查询体重超过均值的学生：

In [ ]:
df.query('Weight > Weight.mean()').head()

#### 【NOTE】query中引用带空格的列名
对于含有空格的列名，需要使用`col name`的方式进行引用。

同时，在`query`中还注册了若干英语的字面用法，帮助提高可读性，例如：`or`,`and`,`in`, `not in`。例如，筛选出男生中不是大一大二的学生：

In [ ]:
df.query('(Grade not in ["Freshman", "Sophomore"]) and (Gender == "Male")').head()

此外，在字符串中出现与列表的比较时，`==`和`!=`分别表示元素出现在列表和没有出现在列表，等价于`in`和`not in`，例如查询所有大三和大四的学生：

In [ ]:
df.query('Grade == ["Junior", "Senior"]').head()

对于`query`中的字符串，如果要引用外部变量，只需在变量名前加`@`符号。例如，取出体重位于70kg到80kg之间的学生：

In [ ]:
low, high = 70, 80
df.query('(Weight >= @low) & (Weight <= @high)').head()

### 6. 随机抽样
如果把`DataFrame`的每一行看作一个样本，或把每一列看作一个特征，再把整个`DataFrame`看作总体，想要对样本或特征进行随机抽样就可以用`sample`函数。有时在拿到大型数据集后，想要对统计特征进行计算来了解数据的大致分布，但是这很费时间。同时，由于许多统计特征在等概率不放回的简单随机抽样条件下，是总体统计特征的无偏估计，比如样本均值和总体均值，那么就可以先从整张表中抽出一部分来做近似估计。

`sample`函数中的主要参数为`n`, `axis`,`frac`,`replace`,`weights`，前三个分别是指抽样数量、抽样的方向（0为行、1为列）和抽样比例（0.3则为从总体中抽出30%的样本）。

`replace`和`weights`分别是指是否放回和每个样本的抽样相对概率，当`replace = True`则表示有放回抽样。例如，对下面构造的`df_sample`以`value`值的相对大小为抽样概率进行有放回抽样，抽样数量为3。

In [ ]:
df_sample = pd.DataFrame({'id': list('abcde'), 'value':[1, 2, 3, 4, 90]})
df_sample

In [ ]:
df_sample.sample(3, replace=True, weights = df_sample.value)

## 二、多级索引
### 1. 多级索引及其表的结构
为了更加清晰地说明具有多级索引的`DataFrame`结构，下面新构造一张表，读者可以忽略这里的构造方法，它们将会在第4小节被更详细地讲解。

In [ ]:
np.random.seed(0)
multi_index = pd.MultiIndex.from_prodcut([list('ABCD'), df.Gender.unique()], names=('School', 'Gender'))
multi_column = pd.MultiIndex.from_product([['Height', 'Weight'], df.Grade.unique()], names=('Idicator', 'Grade'))
df_multi = pd.DataFrame(np.c_[(np.random.randn(8, 4)*5 + 163).tolist(), (np.random.randn(8, 4)*5 + 65).tolist()],
                       index = multi_index, columns=multi_column).round(1)
df_multi

下图通过颜色区分，标记了`DataFrame`的结构。与单层索引的表一样，具备元素值、行索引和列索引三个部分。其中，这里的行索引和列索引都是`MultiIndex`类型，只不过索引中的一个元素是元组而不是单层索引中的标量。例如，行索引的第四个元素为`("B", "Male")`，列索引的第二个元素为`("Height", "Senior")`，这里需要注意，外层连续出现相同的值时，第一次之后出现的会被隐藏显示，使结果的可读性增强。

与单层索引类似，`MultiIndex`也具有名字属性，图中的`School`和`Gender`分别对应了表的第一层和第二层行索引的名字，`Indicator`和`Grade`分别对应了第一层和第二层列索引的名字。

索引的名字和值属性分别可以通过`names`和`values`获得：

In [ ]:
df_multi.index.names

In [ ]:
df_multi.columns.names

In [ ]:
df_multi.index.values

In [ ]:
df_multi.columns.values

如果想要得到某一层的索引，则需要通过`get_level_values`获得：

In [ ]:
df_multi.index.get_level_values(0)

但对于索引而言，无论是单层还是多层，用户都无法通过`index_obj[0] = item`的方式来修改元素，也不能通过`index_name[0] = new_name`的方式来修改名字，关于如何修改这些属性的话题将在第三节被讨论。

### 2. 多级索引中的loc索引器
熟悉了结构后，现在回到原表，将学校和年级设为索引，此时的行为多级索引，列为单级索引，由于默认状态的列索引不含名字，因此对应于刚刚图中`Indicator`和`Grade`的索引名位置是空缺的。